# Ejercicio Módulo 2
**Inteligencia Artificial - CEIA - FIUBA**

**Juan Pablo Skobalski**

En este ejercicio deben implementar un algoritmo de búsqueda que no sea **Búsqueda Primero en Anchura (BFS)** para resolver el problema de la Torre de Hanoi. La nota máxima dependerá del algoritmo implementado:

- **Búsqueda Primero en Profundidad**: nota máxima 6.
- **Búsqueda de Costo Uniforme**: nota máxima 6.
- **Búsqueda de Profundidad Limitada con Profundidad Iterativa**: nota máxima 7.
- **Búsqueda Voraz usando la heurística dada en el aula virtual**: nota máxima 8.
- **Búsqueda Voraz usando una heurística desarrollada por vos**: nota máxima 9.
- **Búsqueda A* usando la heurística dada en el aula virtual**: nota máxima 9.
- **Búsqueda A* usando una heurística desarrollada por vos**: nota máxima 10.

La función debe devolver la salida correspondiente a la solución encontrada o `None si no se encontró una solución.

Además, debe calcular métricas de rendimiento que, como mínimo, incluyan:

- `solution_found`: `True` si se encontró la solución, `False` en caso contrario.
- `nodes_explored`: cantidad de nodos explorados (entero).
- `states_visited`: cantidad de estados distintos visitados (entero).
- `nodes_in_frontier`: cantidad de nodos que quedaron en la frontera al finalizar la ejecución (entero).
- `max_depth`: máxima profundidad explorada (entero).
- `cost_total`: costo total para encontrar la solución (float).

In [22]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from aima_libs.tree_hanoi import NodeHanoi

In [24]:
def search_algorithm(number_disks=5) -> (NodeHanoi, dict):

    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    ##### EDITAR ESTA ZONA

    # Inicializamos las salidas, pero reemplazar con lo que se quiera usar.
    metrics = {
        "solution_found": True,
        "nodes_explored": None,
        "states_visited": None,
        "nodes_in_frontier": None,
        "max_depth": None,
        "cost_total": None,
    }
    solution = NodeHanoi(initial_state)

    # TODO: Completar con el algoritmo de búsqueda que desees implementar
    #####
    """
    En este ejercicio implementé el algoritmo de búsqueda A* para resolver el problema de la Torre de Hanoi, utilizando una heurística propia para garantizar que fuera admisible y consistente.
    El algoritmo mantiene una frontera implementada como una cola de prioridad (heap), ordenada por el valor f(n) = g(n) + h(n), donde g(n) es el costo acumulado y h(n) la estimación heurística (tal como observe en el video de las clase 2.2). Además, guardo un conjunto de estados ya explorados para no reexpandirlos. Cada vez que expando un nodo, genero sus sucesores aplicando las acciones validas.
    """
    
    
    def heuristica_skobalski(estado: StatesHanoi, indice_torre_objetivo: int = 2) -> int:
        pilas = estado.rods
        presentes = [d for pila in pilas for d in pila]
        if not presentes:
            return 0
        
        pos = {}
        for i, pila in enumerate(pilas):
            for d in pila:
                pos[d] = i
        mayor = max(presentes)
        # Desde el más grande hacia el más chico, busco el primero que no esté en la torre objetivo
        for d in range(mayor, 0, -1):
            if pos.get(d, indice_torre_objetivo) != indice_torre_objetivo:
                return (2 ** d) - 1
        return 0

    def codificar(estado: StatesHanoi):
        return tuple(tuple(rod) for rod in estado.rods)

    def actualizar_atributos(nodo: NodeHanoi, padre: NodeHanoi, accion, costo: int):
        nodo.parent = padre
        nodo.action = accion
        nodo.path_cost = costo
        nodo.depth = getattr(padre, "depth", 0) + 1

    # A* con f = g + h; g = pasos (costo u x movimiento), h = heurística skobalski
    import heapq

    nodo_inicial = NodeHanoi(initial_state)
    nodo_inicial.path_cost = 0
    nodo_inicial.depth = 0

    g = {codificar(initial_state): 0}
    h0 = heuristica_skobalski(initial_state, 2)
    frontera = []
    contador = 0  # para desempatar estable en el heap
    heapq.heappush(frontera, (h0, 0, contador, nodo_inicial))
    contador += 1

    cerrados = set()
    visitados = {codificar(initial_state)}
    nodos_explorados = 0
    max_profundidad = 0

    solucion_nodo = None
    costo_final = float("inf")

    while frontera:
        f, g_actual, _, nodo = heapq.heappop(frontera)
        estado = nodo.state
        clave = codificar(estado)
        if clave in cerrados:
            continue

        cerrados.add(clave)
        nodos_explorados += 1
        max_profundidad = max(max_profundidad, getattr(nodo, "depth", g.get(clave, 0)))

        # Goal: uso goal_test si está definido; si no, comparo con el goal_state (tiene __eq__)
        es_goal = problem.goal_test(estado) if hasattr(problem, "goal_test") else (estado == goal_state)
        if es_goal:
            solucion_nodo = nodo
            costo_final = g.get(clave, getattr(nodo, "path_cost", 0))
            break

        # Expansión: actions(state) -> lista de ActionHanoi; result(state, action) -> StatesHanoi
        for accion in problem.actions(estado):
            nuevo_estado = problem.result(estado, accion)
            clave_hijo = codificar(nuevo_estado)
            nuevo_g = g.get(clave, getattr(nodo, "path_cost", 0)) + 1  # costo unitario por movimiento
            if clave_hijo not in g or nuevo_g < g[clave_hijo]:
                g[clave_hijo] = nuevo_g
                visitados.add(clave_hijo)
                hijo = NodeHanoi(nuevo_estado)
                actualizar_atributos(hijo, nodo, accion, nuevo_g)
                h = heuristica_skobalski(nuevo_estado, 2)
                f_nuevo = nuevo_g + h
                heapq.heappush(frontera, (f_nuevo, nuevo_g, contador, hijo))
                contador += 1

    # Métricas y retorno
    if solucion_nodo is not None:
        metrics = {
            "solution_found": True,
            "nodes_explored": nodos_explorados,
            "states_visited": len(visitados),
            "nodes_in_frontier": len(frontera),
            "max_depth": int(max_profundidad),
            "cost_total": float(costo_final),
        }
        solution = solucion_nodo
    else:
        metrics = {
            "solution_found": False,
            "nodes_explored": nodos_explorados,
            "states_visited": len(visitados),
            "nodes_in_frontier": len(frontera),
            "max_depth": int(max_profundidad),
            "cost_total": float("inf"),
        }
        solution = None

    #####

    return solution, metrics

Se prueba la función:

In [25]:
solution, metrics = search_algorithm(number_disks=5)

Veamos las métricas:

In [26]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 116
states_visited: 125
nodes_in_frontier: 9
max_depth: 31
cost_total: 31.0


Veamos el camino de estados desde el principio a la solución:

In [27]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

Y las acciones que el agente debería aplicar para llegar al objetivo:

In [28]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
